In [15]:
%reload_ext autoreload
%autoreload 2

In [11]:
import src.config as config
import hopsworks

In [12]:
# Connect to project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# Connect to feature store
feature_store = project.get_feature_store()

# Connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/42132
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
# Create feature view
# Feature view uses only one feature group

try:
    # Create feature view if one doesn't exist
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )

except:
    print('Feature view already existed. Skip creation...')


# Get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)     

Feature view already existed. Skip creation...


In [8]:
ts_data, _ = feature_view.training_data(
    description="Time series hourly taxi rides"
)

2023-05-08 20:50:05,403 INFO: USE `ml_prod_pipline_featurestore`
2023-05-08 20:50:05,886 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_loc_id` `pickup_loc_id`
FROM `ml_prod_pipline_featurestore`.`ts_hourly_feat_group_1` `fg0`


TypeError: unsupported operand type(s) for *: 'Timestamp' and 'int'

In [ ]:
from src.data import processData2FeatTgt

feats, tgts = processData2FeatTgt(
    ts_data,
    input_feat_len=24*28, # Month
    step_size=23,
)

feats_and_tgts = feats.copy()
feats_and_tgts['tgt_rides_nxt_hr'] = tgts

print(f"{feats_and_tgts.shape=}")

In [ ]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import trainTestSplit

# Training data -> from Jan 2022 up until 2 months ago
# Test Data -> last 2 months

cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1))

print(f"{cutoff_date=}")


X_train, y_train, X_test, y_test = trainTestSplit(
    feats_and_tgts,
    cutoff_date,
    tgt_col_name="tgt_rides_nxt_hr"
)